In [1]:
import sys 
sys.path.append("../")
import pydicom
import os, glob
import misc
import numpy as np
from datetime import datetime

In [2]:
def CalcuateSUV(patientInfor):
    #Method:  SUVbw, SUVlbm or SUVbsa
    #Method='SUVbw';
    Img = np.stack([s.pixel_array for s in patientInfor])
    patientInfor = patientInfor[0]
    
    assert len(patientInfor[0x28, 0x51].value) > 0, 'Corrected Image (0x28,0x51) should contains \
                                                    ATTN and DECAY and Decay Correction (0x0054,0x1102) must be START'
    assert patientInfor[0x54, 0x1102].value == 'START', 'Decay Correction (0x0054,0x1102) must be START'
    assert 'ATTN' and 'DECY' in patientInfor[0x28, 0x51].value, 'Corrected Image (0x0028,0x0051) should contains ATTN and DECAY'
   
    if patientInfor[0x54, 0x1001].value == "BQML": #if Units (0x0054,0x1001) are BQML
        #half life = Radionuclide Half Life (0x0018,0x1075) in Radiopharmaceutical Information Sequence (0x0054,0x0016)
        T_half = 109.8*60 #patient_PET[0][0x54, 0x16][0][0x18, 0x1075].value #half life of FDG

        
        AcquisitionDateandTime = str(int(float(patientInfor.AcquisitionDate + patientInfor.AcquisitionTime)))
        SeriesDateandTime = str(int(float(patientInfor.SeriesDate + patientInfor.SeriesTime)))

        """
        if Series Date (0x0008,0x0021) and Time (0x0008,0x0031) 
        are not after Acquisition Date (0x0008,0x0022) and Time (0x0008,0x0032) 
         """
        if float(SeriesDateandTime) <= float(AcquisitionDateandTime):        
            ScanDateandTime = datetime.strptime(SeriesDateandTime, "%Y%m%d%H%M%S")  #scan Date and Time = Series Date and Time
        else: 
            """may be post-processed series in which Series Date and Time are date of series creation unrelated to acquisition""" 
            if  patientInfor[0x09, 0x10].value == 'GEMS_PETD_01':#if  GE private scan Date and Time (0x0009,0x100d,¡°GEMS_PETD_01¡±) present {
                ScanDateandTime = patientInfor[0x09, 0x100d].value #scan Date and Time = GE private scan Date and Time (0x0009,0x100d,¡°GEMS_PETD_01¡±)        
            else: 
                """
                % // else may be Siemens series with altered Series Date and Time
                % // either check earliest of all images in series (for all bed positions) (wrong for case of PETsyngo 3.x multi-injection)
                % scan Date and Time = earliest Acquisition Date (0x0008,0x0022) and Time (0x0008,0x0032)  in all images of series
                % or
                % // back compute from center (average count rate ) of time window for bed position (frame) in series (reliable in all cases)
                % // Acquisition Date (0x0008,0x0022) and Time (0x0008,0x0032) are the start of the bed position (frame)
                % // Frame Reference Time (0x0054,0x1300) is the offset (ms) from the scan Date and Time we want to the average count rate time
                % if  (Frame Reference Time (0x0054,0x1300) > 0 && Actual Frame Duration (0018,1242) > 0) {
                % frame duration = Actual Frame Duration (0018,1242) / 1000		// DICOM is in ms; want seconds
                % decay constant = ln(2) /  half life
                % decay during frame = decay constant * frame duration
                % average count rate time within frame = 1/decay constant * ln(decay during frame / (1 ¨C exp(-decay during frame)))
                % scan Date and Time = Acquisition Date (0x0008,0x0022) and Time (0x0008,0x0032)
                % -	Frame Reference Time (0x0054,0x1300) /1000 + average count rate time within frame
                % 
                """
                pass
            
        #start Time = Radiopharmaceutical Start Time (0x0018,0x1072) in Radiopharmaceutical Information Sequence (0x0054,0x0016) 
        startTime = patientInfor[0x54, 0x16][0][0x18, 0x1072].value
        StartDateandTime = str(int(float(patientInfor.SeriesDate + startTime)))
        StartDateandTime = datetime.strptime(StartDateandTime, "%Y%m%d%H%M%S")
        decayTime = ScanDateandTime - StartDateandTime 
        total_seconds = decayTime.seconds # decay Time = scan Time ¨C start Time 	// seconds
        #injected Dose = Radionuclide Total Dose (0x0018,0x1074) in Radiopharmaceutical Information Sequence (0x0054,0x0016)	// Bq
        injectedDose = float(patientInfor[0x54, 0x16][0][0x18, 0x1074].value)
        print(injectedDose)
        decayedDose = injectedDose*np.exp(total_seconds*np.log(2)/T_half)    #injectedDose * pow (2, -decayTime / T_half);
        weight = float(patientInfor[0x10, 0x1030].value)
        SUVbwScaleFactor = (weight * 1000 / decayedDose)
   
    elif patientInfor[0x54, 0x1001].value == 'CNTS':# if Units (0x0054,0x1001) are CNTS
        """        
        Philips private scale factor (0x7053,0x1000,¡° Philips PET Private Group¡±)
        if (0x7053,0x1000) not present, but (0x7053,0x1009) is present, then (0x7053,0x1009) * Rescale Slope
        scales pixels to Bq/ml, and proceed as if Units are BQML
        """

        if len(patientInfor[0x7053, 0x1000].value) == 0:
            RescaleSlope=patientInfor.RescaleSlope; #Rescale Slope (0x0028,0x1053)
            SUVbwScaleFactor = patientInfor[0x7053, 0x1009].value * RescaleSlope;
        else:
            SUVbwScaleFactor = patientInfor[0x7053, 0x1000].value 
    elif patientInfor[0x54, 0x1001].value == 'GML': #  if Units (0x0054,0x1001) are GML    
        SUVbwScaleFactor = 1.0 #assumes that GML indicates SUVbw instead of SUVlbm

    RescaleIntercept = patientInfor[0x28, 0x1052].value
    RescaleSlope =  patientInfor[0x28, 0x1053].value
    print(RescaleIntercept, RescaleSlope, SUVbwScaleFactor)
    SUV = (Img + RescaleIntercept )*RescaleSlope*SUVbwScaleFactor
    return SUV

In [3]:
Path = "D:\\LungCancer\\2021-lung-cancer\Data_label_CT_PET_n_256\\AIDATA_PET_20201105(n=246)_20210202+÷-ñ\\"
can_not_cal_suv = []
for dirName, subdirList, fileList in os.walk(Path):
    if (len(fileList) > 1) :
#         if 'LC00588' not in dirName:
#             continue
        patient_PET = misc.load_scan(dirName)
        print(dirName)
        print(patient_PET[0])
        try:
            pet_nor = CalcuateSUV(patient_PET)
            print(pet_nor.min(), pet_nor.max())
        except:
            can_not_cal_suv.append(dirName[-7:])
        print()
        break
        

D:\LungCancer\2021-lung-cancer\Data_label_CT_PET_n_256\AIDATA_PET_20201105(n=246)_20210202+÷-ñ\LC00026
Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 196
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: Positron Emission Tomography Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.2.840.113619.2.379.114374074503672.7872.1589332794165.377
(0002, 0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.840.113619.6.379
(0002, 0013) Implementation Version Name         SH: 'AIA 5.2 (086)'
-------------------------------------------------
(0008, 0000) Group Length                        UL: 446
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY']
(0008, 0012) Instance Creation Date         

In [ ]:
import matplotlib.pyplot as plt
for i in range(pet_nor.shape[0]):
    plt.imshow(pet_nor[i, :, :])
    plt.show()

In [ ]:
pet_nor.min(), pet_nor.max()

In [ ]:
x = 3**0.6
16*x**5 - 15*x**4 - 18*x**3 - 8*x**2 +18*x -9